# Music ROI Summary
Create ROI-level summaries for music (mean z per atlas region, per subject) and export them as
CSV tables.

In [1]:
# Cell 3 – paths and subject list

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nilearn import image, masking, plotting, datasets

# --- project paths ---
PROJECT_ROOT = Path(".")
DERIV_ROOT   = PROJECT_ROOT / "derivatives" / "music"
FIRSTLEVEL_ROOT = DERIV_ROOT / "firstlevel"

# subjects used in music project (run-02)
MUSIC_SUBJECTS = ["sub-001", "sub-002", "sub-003", "sub-004", "sub-005"]

print("DERIV_ROOT   :", DERIV_ROOT)
print("FIRSTLEVEL_ROOT:", FIRSTLEVEL_ROOT)
print("Subjects     :", MUSIC_SUBJECTS)

DERIV_ROOT   : derivatives\music
FIRSTLEVEL_ROOT: derivatives\music\firstlevel
Subjects     : ['sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub-005']


In [2]:
# Cell 4 – list scene/contrast z-maps and set up atlas choice

# z-maps we expect from Phase 4
for sub in MUSIC_SUBJECTS:
    z_path = FIRSTLEVEL_ROOT / f"{sub}_run-02_music_zmap.nii.gz"
    print(f"{sub}: {z_path}  (exists: {z_path.exists()})")

# We'll use Harvard–Oxford cortical atlas via Nilearn
atlas_ho = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_img_path = atlas_ho.maps
atlas_labels   = list(atlas_ho.labels)

print("\nAtlas image:", atlas_img_path)
print("Number of atlas labels:", len(atlas_labels))
print("First 10 labels:", atlas_labels[:10])

# Load atlas image
atlas_img = image.load_img(atlas_img_path)
atlas_data = atlas_img.get_fdata().astype(int)

sub-001: derivatives\music\firstlevel\sub-001_run-02_music_zmap.nii.gz  (exists: True)
sub-002: derivatives\music\firstlevel\sub-002_run-02_music_zmap.nii.gz  (exists: False)
sub-003: derivatives\music\firstlevel\sub-003_run-02_music_zmap.nii.gz  (exists: False)
sub-004: derivatives\music\firstlevel\sub-004_run-02_music_zmap.nii.gz  (exists: False)
sub-005: derivatives\music\firstlevel\sub-005_run-02_music_zmap.nii.gz  (exists: False)


[fetch_atlas_harvard_oxford] Dataset created in C:\Users\Nirwaan Azhar\nilearn_data\fsl

[fetch_atlas_harvard_oxford] Downloading data from https://www.nitrc.org/frs/download.php/9902/HarvardOxford.tgz 
...

[fetch_atlas_harvard_oxford] Downloaded 11837440 of 25716861 bytes (46.0%%,    1.2s remaining)

[fetch_atlas_harvard_oxford]  ...done. (2 seconds, 0 min)

[fetch_atlas_harvard_oxford] Extracting data from C:\Users\Nirwaan 
Azhar\nilearn_data\fsl\25fa7171ea28e260d13aae6b2a9ea411\HarvardOxford.tgz...

[fetch_atlas_harvard_oxford] .. done.


Atlas image: 
<class 'nibabel.nifti1.Nifti1Image'>
data shape (91, 109, 91)
affine:
[[   2.    0.    0.  -90.]
 [   0.    2.    0. -126.]
 [   0.    0.    2.  -72.]
 [   0.    0.    0.    1.]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3  91 109  91   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [1. 2. 2. 2. 1. 1. 1. 1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 48.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'FSL3.3'
aux_file        : b'MGH-Cortical'
qform_code      :

In [3]:
# Cell 5 – build ROI index/name table (skip background)

roi_indices = []
roi_names = []
for idx, name in enumerate(atlas_labels):
    if idx == 0:
        continue
    roi_indices.append(idx)
    roi_names.append(name)

roi_table = pd.DataFrame({"roi_index": roi_indices, "roi_name": roi_names})
print("Number of ROIs (excluding background):", len(roi_indices))
roi_table.head(10)

Number of ROIs (excluding background): 48


,roi_index,roi_name
0,1,Frontal Pole
1,2,Insular Cortex
2,3,Superior Frontal Gyrus
3,4,Middle Frontal Gyrus
4,5,"Inferior Frontal Gyrus, pars triangularis"
5,6,"Inferior Frontal Gyrus, pars opercularis"
6,7,Precentral Gyrus
7,8,Temporal Pole
8,9,"Superior Temporal Gyrus, anterior division"
9,10,"Superior Temporal Gyrus, posterior division"


In [4]:
# Cell 6 – helper function: ROI means for one subject z-map

def extract_roi_means(z_map_img, atlas_img, roi_indices, roi_names):
    """
    Compute mean z-score in each ROI for a given z-map.
    Returns a pandas Series with index = roi_names.
    """
    atlas_resampled = image.resample_to_img(atlas_img, z_map_img, interpolation="nearest")
    atlas_resampled_data = atlas_resampled.get_fdata().astype(int)
    z_data = z_map_img.get_fdata()

    roi_means = []
    for idx in roi_indices:
        mask = atlas_resampled_data == idx
        if not np.any(mask):
            roi_means.append(np.nan)
            continue
        vals = z_data[mask]
        vals = vals[np.isfinite(vals)]
        roi_means.append(vals.mean() if vals.size > 0 else np.nan)

    return pd.Series(roi_means, index=roi_names)

In [5]:
# Cell 7 – loop over subjects and build wide + long ROI tables

all_roi_rows = []
long_rows = []

for sub in MUSIC_SUBJECTS:
    z_path = FIRSTLEVEL_ROOT / f"{sub}_run-02_music_zmap.nii.gz"
    if not z_path.exists():
        print(f"[WARN] Missing z-map for {sub}: {z_path}")
        continue

    z_img = image.load_img(z_path)
    roi_series = extract_roi_means(z_img, atlas_img, roi_indices=roi_indices, roi_names=roi_names)

    # wide row
    row_dict = {"subject": sub}
    row_dict.update(roi_series.to_dict())
    all_roi_rows.append(row_dict)

    # long rows
    for roi_name, val in roi_series.items():
        long_rows.append({"subject": sub, "ROI": roi_name, "mean_z": val})

# wide format
music_roi_wide = pd.DataFrame(all_roi_rows).set_index("subject")
print("Wide ROI table shape:", music_roi_wide.shape)

# long format
music_roi_long = pd.DataFrame(long_rows)
print("Long ROI table shape:", music_roi_long.shape)

# display small snippets
music_roi_wide.head(), music_roi_long.head()

[WARN] Missing z-map for sub-002: derivatives\music\firstlevel\sub-002_run-02_music_zmap.nii.gz
[WARN] Missing z-map for sub-003: derivatives\music\firstlevel\sub-003_run-02_music_zmap.nii.gz
[WARN] Missing z-map for sub-004: derivatives\music\firstlevel\sub-004_run-02_music_zmap.nii.gz
[WARN] Missing z-map for sub-005: derivatives\music\firstlevel\sub-005_run-02_music_zmap.nii.gz
Wide ROI table shape: (1, 48)
Long ROI table shape: (48, 3)


C:\Users\Nirwaan Azhar\AppData\Local\Temp\ipykernel_35436\3578710881.py:8: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  atlas_resampled = image.resample_to_img(atlas_img, z_map_img, interpolation="nearest")
C:\Users\Nirwaan Azhar\AppData\Local\Temp\ipykernel_35436\3578710881.py:8: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  atlas_resampled = image.resample_to_img(atlas_img, z_map_img, interpolation="nearest")


(         Frontal Pole  Insular Cortex  Superior Frontal Gyrus  \
 subject                                                         
 sub-001     -0.021414       -0.083435                0.002456   
 
          Middle Frontal Gyrus  Inferior Frontal Gyrus, pars triangularis  \
 subject                                                                    
 sub-001              0.001332                                   0.016991   
 
          Inferior Frontal Gyrus, pars opercularis  Precentral Gyrus  \
 subject                                                               
 sub-001                                  0.110271         -0.214885   
 
          Temporal Pole  Superior Temporal Gyrus, anterior division  \
 subject                                                              
 sub-001      -0.013861                                   -0.060834   
 
          Superior Temporal Gyrus, posterior division  ...  \
 subject                                               ...   
 sub-001  

In [6]:
# Cell 8 – save ROI tables to disk

ROI_ROOT = DERIV_ROOT / "roi"
ROI_ROOT.mkdir(parents=True, exist_ok=True)

wide_path = ROI_ROOT / "music_roi_means_wide.csv"
long_path = ROI_ROOT / "music_roi_means_long.csv"

music_roi_wide.to_csv(wide_path)
music_roi_long.to_csv(long_path, index=False)

print("Saved wide ROI table to :", wide_path)
print("Saved long ROI table to :", long_path)

Saved wide ROI table to : derivatives\music\roi\music_roi_means_wide.csv
Saved long ROI table to : derivatives\music\roi\music_roi_means_long.csv


In [ ]:
# Cell 9 – group mean per ROI + barplot of top ROIs

# group mean across subjects (ignoring NaNs)
group_mean_roi = music_roi_wide.mean(axis=0)

group_mean_df = (
    group_mean_roi
    .rename("group_mean_z")
    .reset_index()
    .rename(columns={"index": "ROI"})
    .sort_values("group_mean_z", ascending=False)
)

print("Top 10 ROIs by group mean z:")
group_mean_df.head(10)

# Barplot of top 10 ROIs
top_n = 10
top_df = group_mean_df.head(top_n)

plt.figure(figsize=(8, 5))
plt.barh(top_df["ROI"], top_df["group_mean_z"])
plt.gca().invert_yaxis()   # largest on top
plt.xlabel("Mean z (music)")
plt.title("Top ROIs for Music Activation (mean z across subjects)")
plt.tight_layout()
plt.show()

In [ ]:
# Cell 10 – optional: heatmap of subjects × ROIs for top ROIs

import seaborn as sns

# Select top ROIs by group mean
top_rois = group_mean_df.head(15)["ROI"].tolist()
music_roi_wide_top = music_roi_wide[top_rois]

plt.figure(figsize=(10, 4))
sns.heatmap(music_roi_wide_top, cmap="RdBu_r", center=0, cbar_kws={"label": "Mean z"})
plt.title("Music activation heatmap: subjects × top ROIs")
plt.ylabel("Subject")
plt.xlabel("ROI")
plt.tight_layout()
plt.show()